In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from time import gmtime, strftime
import sys
import os
import io

import string
import re
import itertools
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/julie.fisher/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv('../../github/icij_investigation/aws/data/edited/addresses_Bahamas.csv')

In [3]:
df.head()

,node_id,address,name,countries,country_codes,sourceID,valid_until,note
0,24000001,"ANNEX FREDERICK & SHIRLEY STS, P.O. BOX N-4805...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN
1,24000002,"SUITE E-2,UNION COURT BUILDING, P.O. BOX N-818...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN
2,24000003,"LYFORD CAY HOUSE, LYFORD CAY, P.O. BOX N-7785,...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN
3,24000004,"P.O. BOX N-3708 BAHAMAS FINANCIAL CENTRE, P.O....",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN
4,24000005,"LYFORD CAY HOUSE, 3RD FLOOR, LYFORD CAY, P.O. ...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN


## To do

- [x] Standardize "&" to "and"
- [x] Lowercase everything
- [x] Remove puncutation
- [x] Standardize street names/abbreviations
- [x] Word counts
- Fuzzy matching

In [4]:
pd.set_option('display.max_colwidth', 1000)

## Create a new column for making changes

It's important to keep a copy of the original values. This makes it much easier to revert changes that don't perform as expected and also to reference if data is lost or to put it back into context.

In [5]:
df['working_address'] = df['address']

In [6]:
df['working_address']

0                                                                                   ANNEX FREDERICK & SHIRLEY STS, P.O. BOX N-4805, NASSAU, BAHAMAS
1                                                                                  SUITE E-2,UNION COURT BUILDING, P.O. BOX N-8188, NASSAU, BAHAMAS
2                                                                                    LYFORD CAY HOUSE, LYFORD CAY, P.O. BOX N-7785, NASSAU, BAHAMAS
3                                                P.O. BOX N-3708 BAHAMAS FINANCIAL CENTRE, P.O. BOX N-3708 SHIRLEY & CHARLOTTE STS, NASSAU, BAHAMAS
4                                                                         LYFORD CAY HOUSE, 3RD FLOOR, LYFORD CAY, P.O. BOX N-3024, NASSAU, BAHAMAS
                                                                           ...                                                                     
2253                                                                        J.P.MORGAN TRUST COMPANY (BAHAMAS) L

## Standardize values

Most matching algorithms are based on exact matches. Small changes can make using these common matching functions useless. Such changes include abbreviations like "st" and "blvd" for "street" and "boulevard", capitalization differences such as "Annex Frederick" vs "ANNEX FREDERICK", and punctuation such as "P.O. Box" vs "PO Box". As a general rule, the use of multiple representations for the same thing makes it harder to match like values.

Standardizing datasets makes finding patterns and trends much easier.

### Replace "&"

In the Bahamas addresses "&" and "and" are used interchangeably.

I did a quick sanity check to ensure that "&" wasn't being used in another way. There are only 326 rows that use "&". A quick perusal shows that most, if not all, are used to connect street names.

In [7]:
df.loc[df['working_address'].str.contains("&"), 'working_address']

0                                                                                  ANNEX FREDERICK & SHIRLEY STS, P.O. BOX N-4805, NASSAU, BAHAMAS
3                                               P.O. BOX N-3708 BAHAMAS FINANCIAL CENTRE, P.O. BOX N-3708 SHIRLEY & CHARLOTTE STS, NASSAU, BAHAMAS
8                                                                       BAYSIDE EXECUTIVE PARK, WEST BAY & BLAKE, P.O. BOX N-4875, NASSAU, BAHAMAS
10                                                      TK HOUSE, BAYSIDE EXECUTIVE PARK, P.O. BOX AP-59213 WEST BAY & BLAKE ROAD, NASSAU, BAHAMAS
11                                                                         BAYSIDE HOUSE WEST BAY & BLAKE ROAD, P.O. BOX AP-59213, NASSAU, BAHAMAS
                                                                           ...                                                                    
2213                                                                    MARRON HOUSE, P.0. BOX N-4826, VIRGINIA & AUGU

### Lowercase everything

Different capitalization strategies quickly complicate an analysis as most value matching is based on exact matches.

In [8]:
df['working_address'] = df['address'].str.replace('&', 'and').str.lower()

In [9]:
df.loc[df['working_address'].str.contains("&"), 'working_address']

Series([], Name: working_address, dtype: object)

In [10]:
df['working_address']

0                                                                                 annex frederick and shirley sts, p.o. box n-4805, nassau, bahamas
1                                                                                  suite e-2,union court building, p.o. box n-8188, nassau, bahamas
2                                                                                    lyford cay house, lyford cay, p.o. box n-7785, nassau, bahamas
3                                              p.o. box n-3708 bahamas financial centre, p.o. box n-3708 shirley and charlotte sts, nassau, bahamas
4                                                                         lyford cay house, 3rd floor, lyford cay, p.o. box n-3024, nassau, bahamas
                                                                           ...                                                                     
2253                                                                        j.p.morgan trust company (bahamas) l

### Remove punctuation

Punctuation can be particularly helpful in splitting fomratted text blocks into smaller pieces. However, there is no standardized format for these addresses, thus the punctuation actually makes pulling out relevant information harder in this dataset. As such, I'm going to get rid of it.

In [11]:
newline_list = '\t\r\n'
remove_newline = str.maketrans(' ', ' ', newline_list)
punct_list = string.punctuation
nopunct = str.maketrans('', '', punct_list)

In [12]:
df['working_address'] = df['working_address'].str.translate(remove_newline).str.translate(nopunct)
df['working_address']

0                                                                            annex frederick and shirley sts po box n4805 nassau bahamas
1                                                                               suite e2union court building po box n8188 nassau bahamas
2                                                                                lyford cay house lyford cay po box n7785 nassau bahamas
3                                            po box n3708 bahamas financial centre po box n3708 shirley and charlotte sts nassau bahamas
4                                                                      lyford cay house 3rd floor lyford cay po box n3024 nassau bahamas
                                                                      ...                                                               
2253                                                                      jpmorgan trust company bahamas limited nassau n4899 zh bahamas
2254                                     

### Standardize abbreviations

I got the majority of the abbreviations the hard way, I went though the dataset by hand. I was attempting to pull out the city name for each address that had a street in it. The only way to do this for many addresses was to look at the street, resulting in a lot of `contains` searches.

In the frequency count I also noticed that "p" and "o" occur rather frequently. A quick peek shows that in 84 rows "po box" is listed as "p o box". When doing replacements like these, it's important to do sanity checks as the results won't always be what you expect. One of my favorite examples was a search for "demon", I also got "demonstrate". Watch out for these kinds of things. Fortunately, in the "po box" example for the Bahamas addresses "p o" only occurs when for PO boxes.

There are also occurences of "pobox", but in joining "p" and "o" from "p o" gives me one last "pobox" so I'll need to apply this after the main abbreviation changes.

In [13]:
df.loc[df['working_address'].str.contains('p o'), 'working_address']

650                                                                              bitco building third floor p o box n8188 nassau bahamas
655                                                                  bolam house king and george streets p o box cb 11343 nassau bahamas
788                                                   landl corporate and legal services east bay street p o box ss 19812 nassau bahamas
801                                                                           malborough and queens street p o box n10429 nassau bahamas
818     morgan trust co of bahamas ltd the bahamas financial center 2nd floor charlotte and shorley street p o box n4899  nassau bahamas
                                                                      ...                                                               
2140                                                     montague sterling centre 3rd floor east bay street p o box n3242 nassau bahamas
2163                                     

In [14]:
df.loc[df['working_address'].str.contains('pobox'), 'working_address']

564                                                                              50 shirley streetpobox cb 13937 nassau bahamas
702                                          citibank building thompson boulevard pobox n1576 nassau new providence the bahamas
749                           fourth floor the bahamas financial centre shirley and charlotte streets pobox n3023 nassau bahmas
826     mossack fonseca and co bahams limited saffrey square suite 205 bank lane pobox n8188 nassau commonwealth of the bahamas
856                                                                                                  pobox n1491 nassau bahamas
867                                                                 pobox n3944 providence house east hill street nassau bahams
870                                                                                                 pobox n 4875 nassau bahamas
1025                               suite e2 union court building elizabeth avenue and shirley street pob

#### The difficulties of "street"

Speaking of tricking searches, "st" was particularly interesting. Simply replacing "st" with "street" will also alter the following words:

- street: streetreet
- sts: streets
- west: westreet
- east: eastreet
- st: street

I compromised to look for " st " esentially ensuring there was a space before and after "st".

In [16]:
df.loc[df['working_address'].str.contains('(\sst\s)',regex=True), 'working_address']

<ipython-input-16-0a12ecad03d6>:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df.loc[df['working_address'].str.contains('(\sst\s)',regex=True), 'working_address']


7                                      providence house east wing east hill st po box cb12399 nassau bahamas
12                                                     308 east bay st 4th floor po box n7768 nassau bahamas
16                                    sassoon house shirley st and victoria ave po box ss5383 nassau bahamas
21                                      3rd floor one montague place east bay st po box n3231 nassau bahamas
34                                         31b annex building east bay st 2nd fl po box n3930 nassau bahamas
                                                        ...                                                 
2093                                     goodmans corporate center west bay st po box sp61567 nassau bahamas
2096                                   providence house east wing east hill st po box cb12399 nassau bahamas
2097                                  sassoon house shirley st and victoria ave po box ss5383 nassau bahamas
2104               

In [17]:
abbrev_dict = {'\sst\s': 'street',
              'str\s': 'street',
              'streets': 'street',
              'sts': 'street',
              'blvd': 'boulevard',
              'sq\s': 'square',
              'dr\s': 'drive',
              'ave\s': 'avenue',
              '\sln': 'lane',
              'lanes': 'lane',
              'hwy': 'highway',
              '1st': 'first',
              '2nd': 'second',
              '2 nd': 'second',
              '3rd': 'third',
              '4th': 'fourth',
              '5th': 'fifth',
              '6th': 'sixth',
              '7th': 'seventh',
              '8th': 'eighth',
              '9th': 'ninth',
              'p o': 'po'}

In [18]:
df['working_address'] = df['working_address'].replace(abbrev_dict, regex=True)

In [19]:
df['working_address'] = df['working_address'].replace('pobox', 'po box')

## Count words

I frequently use word counts to determine any words I need to add to the stopword list (stopwords are words that occur frequently, such as 'the' or 'a' that don't add value to the analysis and should be removed).

In this case, I'm hoping it will bubble up things like cities, states, islands, and other address features that occur frequently in the data. I'm filling blank lines in with an empty string to help full dataset processing (i.e. I really don't want to deal with NANs).

In [20]:
df['address_wordlist'] = df['working_address'].fillna('').str.split()

In [21]:
def frequency_ct(ngram_list):
    freq_dict = {}
    for ngram in ngram_list:
        if ngram not in freq_dict:
            freq_dict[ngram] = 0
        freq_dict[ngram] +=1
    return freq_dict

In [22]:
freq_df = pd.DataFrame.from_dict(
    frequency_ct(df['address_wordlist'].sum()
                ), orient='index').reset_index().rename(
    columns={'index':'word', 0:'count'}).sort_values('count', ascending=False)

In [23]:
freq_df.head(60)

,word,count
9,bahamas,2311
8,nassau,2009
6,box,1451
5,po,1402
4,street,1045
2,and,612
3,shirley,477
10,suite,447
33,bay,397
13,building,326


## Fuzzy matching